# The Helmert transform
$
\begin{bmatrix}
    x\\
    y\\
    z\\
\end{bmatrix}^B
=
\begin{bmatrix}
    t_X\\
    t_Y\\
    t_Z\\
\end{bmatrix}
+
\begin{bmatrix}
    1+s  & -r_Z & r_Y\\
    r_Z  & 1+s  & -r_X\\
    -r_Y & r_X  & 1+s\\
\end{bmatrix}
\cdot
\begin{bmatrix}
    x\\
    y\\
    z\\
\end{bmatrix}^A
$

### Using an external Rust library to speed up lon, lat to [BNG](https://en.wikipedia.org/wiki/Ordnance_Survey_National_Grid) conversion

In [9]:
import numpy as np
import pandas as pd
import math
from ctypes import cdll, c_float, c_double, Structure, ARRAY, POINTER, c_int32, c_uint32, c_size_t, c_void_p, cast
from sys import platform
from bng import bng
import pyproj
import ipdb
from array import array

In [10]:
def helmert(x, y, z=0):
    """ Example implementation of Helmert transform """
    tX = -446.448  
    tY = 125.157  
    tZ = -542.060  

    rX = -0.1502  
    rY = -0.2470  
    rZ = -0.8421  

    s = 20.4894 * math.pow(10, -6)
    # For multiple x and y
    # A_vector = np.matrix([[x, y, z], [x, y, z]]).T
    A_vector = np.vstack(np.array([x, y, z]))
    t_matrix = np.vstack(np.array([tX, tY, tZ]))
    conversion = np.matrix([[1 + s, -rZ, rY], [rZ, 1 + s, -rX], [-rY, rX, 1 + s]])
    return t_matrix + (conversion * A_vector)

In [11]:
helmert(-2.018304, 54.589097)

matrix([[-402.49686677],
        [ 181.4468293 ],
        [-550.75780346]])

### Setting up the Rust library. See [here](https://github.com/alexcrichton/rust-ffi-examples/tree/master/python-to-rust) for more

Ensure you've built your Rust library using `cargo build --release`, or the next step will fail.

The boilerplate below can easily be hidden in a wrapper function – it's just here to demonstrate how to call into a shared Rust lib using FFI.

In [12]:
if platform == "darwin":
    ext = "dylib"
else:
    ext = "so"
    
lib = cdll.LoadLibrary('target/release/liblonlat_bng.' + ext)

Define the `ctypes` structures for lon, lat --> BNG conversion

In [13]:
class _FFIArray(Structure):
    """ Convert sequence of floats to a C-compatible void array """
    _fields_ = [("data", c_void_p),
                ("len", c_size_t)]

    @classmethod
    def from_param(cls, seq):
        """  Allow implicit conversions from a sequence of 64-bit floats."""
        return seq if isinstance(seq, cls) else cls(seq)

    def __init__(self, seq, data_type = c_double):
        """
        Convert sequence of values into array, then ctypes Structure

        Rather than checking types (bad), we just try to blam seq
        into a ctypes object using from_buffer. If that doesn't work,
        we try successively more conservative approaches:
        numpy array -> array.array -> read-only buffer -> CPython iterable
        """
        try:
            len(seq)
        except TypeError:
             # we've got an iterator or a generator, so consume it
            seq = array('d', seq)
        array_type = data_type * len(seq)
        try:
            raw_seq = array_type.from_buffer(seq.astype(np.float64))
        except (TypeError, AttributeError):
            try:
                raw_seq = array_type.from_buffer_copy(seq.astype(np.float64))
            except (TypeError, AttributeError):
                # it's a list or a tuple
                raw_seq = array_type.from_buffer(array('d', seq))
        self.data = cast(raw_seq, c_void_p)
        self.len = len(seq)


class _Result_Tuple(Structure):
    """ Container for returned FFI data """
    _fields_ = [("e", _FFIArray),
                ("n", _FFIArray)]


def _void_array_to_list(restuple, _func, _args):
    """ Convert the FFI result to Python data structures """
    eastings = POINTER(c_double * restuple.e.len).from_buffer_copy(restuple.e)[0]
    northings = POINTER(c_double * restuple.n.len).from_buffer_copy(restuple.n)[0]
    res_list = [list(eastings), list(northings)]
    drop_array(restuple.e, restuple.n)
    return res_list

Define `ctypes` input and return parameters

In [14]:
# Multi-threaded FFI functions
convert_bng = lib.convert_to_bng_threaded_nocopy
convert_bng.argtypes = (_FFIArray, _FFIArray)
convert_bng.restype = _Result_Tuple
convert_bng.errcheck = _void_array_to_list

convert_lonlat = lib.convert_to_lonlat_threaded
convert_lonlat.argtypes = (_FFIArray, _FFIArray)
convert_lonlat.restype = _Result_Tuple
convert_lonlat.errcheck = _void_array_to_list

convert_to_osgb36_threaded = lib.convert_to_osgb36_threaded
convert_to_osgb36_threaded.argtypes = (_FFIArray, _FFIArray)
convert_to_osgb36_threaded.restype = _Result_Tuple
convert_to_osgb36_threaded.errcheck = _void_array_to_list

convert_osgb36_to_ll = lib.convert_osgb36_to_ll_threaded
convert_osgb36_to_ll.argtypes = (_FFIArray, _FFIArray)
convert_osgb36_to_ll.restype = _Result_Tuple
convert_osgb36_to_ll.errcheck = _void_array_to_list

convert_etrs89_en_to_osgb36 = lib.convert_etrs89_to_osgb36_threaded
convert_etrs89_en_to_osgb36.argtypes = (_FFIArray, _FFIArray)
convert_etrs89_en_to_osgb36.restype = _Result_Tuple
convert_etrs89_en_to_osgb36.errcheck = _void_array_to_list

convert_osgb36_en_to_etrs89 = lib.convert_osgb36_to_etrs89_threaded
convert_osgb36_en_to_etrs89.argtypes = (_FFIArray, _FFIArray)
convert_osgb36_en_to_etrs89.restype = _Result_Tuple
convert_osgb36_en_to_etrs89.errcheck = _void_array_to_list

# Free FFI-allocated memory
drop_array = lib.drop_float_array
drop_array.argtypes = (_FFIArray, _FFIArray)
drop_array.restype = None

## Simple test of average conversion speed, Python version

## Test: 1MM random points within the UK

In [15]:
# UK bounding box
N = 55.811741
E = 1.768960
S = 49.871159
W = -6.379880

bng = pyproj.Proj(init='epsg:27700')
wgs84 = pyproj.Proj(init='epsg:4326')

num_coords = 1000000
lon_ls = list(np.random.uniform(W, E, [num_coords]))
lat_ls = list(np.random.uniform(S, N, [num_coords]))

### Pure Python

In [ ]:
%%timeit -r50
[bng(lat, lon) for lat, lon in zip(lat_ls, lon_ls)]

### Pyproj

In [10]:
%%timeit -r50
zip(*pyproj.transform(wgs84, bng, lon_ls, lat_ls))

1 loop, best of 50: 537 ms per loop


### Multithreaded Rust

In [16]:
%%timeit -r50
zip(*convert_bng(lon_ls, lat_ls))

1 loop, best of 50: 624 ms per loop


## Pyproj is now only 1.2x (~20%) faster than Multithreaded Rust, which is 8–9x faster than pure Python

## Ignore this for now

In [18]:
df = pd.DataFrame({
        'lons': lon_ls,
        'lats': lat_ls
        })

In [25]:
%%timeit -r25
df['eastings'], df['northings'] = convertbng_threaded(df.lons.values, df.lats.values)
df.head()

1 loop, best of 25: 903 ms per loop


In [8]:
convert_bng([-2.0183041005533306], [54.589097162646141])

[[398915.033], [521545.067]]